# 전처리 모듈: preprocess.py
* 주피터노트북이 존재하는 경로 상에 preprocess.py 파일이 있어야 함

In [ ]:
# 경로 설정   
import os
os.getcwd()

In [ ]:
import pandas as pd 
import numpy as np
import datetime as dt
from  datetime import datetime, timedelta, date
from math import sqrt

from sklearn.linear_model import LinearRegression
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale
from tqdm import tqdm
import joblib

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## 폰트
from matplotlib import font_manager, rc
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

import time
import pickle

import plotly 
import plotly.express as px
import plotly.graph_objects as go

import warnings
warnings.filterwarnings('ignore')

## display
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_seq_items', None)

In [ ]:
# 전처리 모듈
import preprocess_ver0913 as prep

In [ ]:
import catboost
from catboost import *
import shap

In [ ]:
import xgboost as xgb
from catboost import Pool
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor

from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.feature_selection import RFE,RFECV
from sklearn.naive_bayes import GaussianNB
from mlxtend.regressor import StackingRegressor
from sklearn.linear_model import LinearRegression, Lasso, LassoCV
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import KFold, cross_val_score,StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV 
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_squared_log_error
from sklearn.preprocessing import LabelEncoder

# Model 1. CatBoost

## 데이터 불러오기

In [ ]:
#df_train
df = pd.read_csv('df_ver0914.csv', encoding='cp949-')
df["기준일자"] = pd.to_datetime(df['기준일자'], format='%Y-%m-%d')
df['year_month'] = pd.to_datetime(df['year_month'])


#df_test
df_test = pd.read_csv('df_test_pred.csv', encoding='cp949')
df_test["기준일자"] = pd.to_datetime(df_test['기준일자'], format='%Y-%m-%d')

In [ ]:
df = df[df['배출량'].notna()]

In [ ]:
df['dateName'] = df['dateName'].fillna('일반')
df_test['dateName'] = df_test['dateName'].fillna('일반')
df_test['화물량'] = df_test['화물량'].fillna(9104.84)
df = df.fillna(0)
df_test = df_test.fillna(0)

In [ ]:
drop_list = ['행정동_코드', '배출건수', '총_지불금액', '외식_결제건수', '외식_결제금액', 
       '간식_결제건수', '농축수산물_결제건수', '마트/슈퍼마켓_결제건수', '배달_결제건수', '부페_결제건수',
       '식품_결제건수', '아시아음식_결제건수', '양식_결제건수', '주점및주류판매_결제건수', '패스트푸드_결제건수',
       '한식_결제건수', '간식_결제금액', '농축수산물_결제금액', '마트/슈퍼마켓_결제금액', '배달_결제금액',
       '부페_결제금액', '식품_결제금액', '아시아음식_결제금액', '양식_결제금액', '주점및주류판매_결제금액',
       '패스트푸드_결제금액', '한식_결제금액', '외식_결제건수', '외식_결제금액', '1회당_외식_결제금액',
       '1회당_마트/슈퍼마켓_결제금액', '1회당_배달_결제금액', '1회당_농축수산물_결제금액']
df = df.drop(drop_list, axis=1)

In [ ]:
df_train = df.copy()

## LASSO 변수선택

### 라벨 인코딩

In [ ]:
#cate_fe = {'시_구분','dayofweek_name','season','dateName','period','행정동명','읍면동'}
cate_fe = {'year','dayofweek_name','season','dateName','행정동명','시_구분','period','읍면동','month'}
for col in cate_fe:
    local_map = {}
    for i, loc in enumerate(df[col].unique()):
        local_map[loc] = i
    df[col] = df[col].map(local_map)
    df[col] = df[col].astype('category')

In [ ]:
# 범주형 변수 지정
cat_features = ["시_구분", "행정동명",'dayofweek_name','season','dateName','is_weekend','isHoliday',
                'year','month','clust_방문거주', 'clust_방문거주2',
                'kclus_배출', 'kclus_거주연령', 'kclus_시간유동', 'kclus_방문유동', 'kclus_카드소비', 
                'kclus_ratio','kclus_세대수', 'kclus_배출량2','kclus_카드','kclus_month_코로나',
                'period','읍면동']

In [ ]:
# 피쳐 리스트
features = ['kclus_세대수','kclus_month_코로나','면적','density','읍면동','평균 해면기압(hPa)',
            '30대_항공','30대_맛집','20대_맛집','제주_확진자수','30대_버스','신선식품지수','day','60대_숙박',
            '평균 상대습도(%)','50대_제주코로나','50대_버스','60대_맛집','period','40대_날씨','합계 일사량(MJ/m2)',
            '세대당_농축수산물_결제금액','여성_거주인구','30대_날씨','제주항공주가','전국_격리중_환자수','고등학교등교여부',
            '초등학교등교여부','태풍','제주_사망자수','평균기온(°C)','제주_해외유입수','is_weekend','1시간 최다일사량(MJ/m2)',
            '전체_렌트카','최대 순간 풍속(m/s)','20대_숙박','20대_렌트카','50대_날씨','40대_숙박','세대수','식료품소매',
            'dayofweek_name','dateName','60대_렌트카','20대_코로나','평균 풍속(m/s)','최고 해면기압(hPa)','전국_해외유입수',
            '전체_코로나','60대_제주코로나','중학교개수','40대_맛집','남성_거주인구','isHoliday','행정동명','50대_여성_코로나',
            '30대_렌트카','clust_방문거주','제주_격리중_환자수','평균 전운량(1/10)','40대_여성_코로나','제주_전일대비_증감수',
            '20대_버스','전체_제주코로나','60대_날씨','전체_숙박','kclus_ratio','최저기온(°C)','합계 소형증발량(mm)',
            'kclus_배출','평균 현지기압(hPa)','60대_남성_코로나','50대_맛집','캠프/별장/펜션','세대당_배달_결제건수',
            '합계 대형증발량(mm)','dayofyear','최고기온(°C)','최저 해면기압(hPa)','초등학교개수','전체_항공',
            '평균 중하층운량(1/10)','time_idx','60대_버스','50대_항공','전체_버스','생활물가지수','강수 계속시간(hr)',
            'kclus_결제건수','유스호스텔','대기업본사','최소 상대습도(%)','kclus_카드소비','전국_확진자수','20대_항공',
            'year','20대_제주코로나','60대_여성_코로나','60대_항공','전국_전일대비_증감수','상가','평균 이슬점온도(°C)',
            '학문/교육','month_cos','50대_숙박','숙박','거점지역_개수','민박/하숙','고등학교개수','30대_제주코로나',
            '안개 계속시간(hr)','40대_제주코로나','weeknumber','제주_격리해제수','kclus_배출량2','스타벅스','50대_남성_코로나',
            '40대_항공','30대_숙박','kclus_시간유동','한국인세대수','하나투어주가','일강수량(mm)','캠핑장',
            '세대당_마트/슈퍼마켓_결제금액','50대_렌트카','10분 최다 강수량(mm)','month','1시간 최다강수량(mm)',
            '최대 풍속(m/s)','20대_날씨','합계 일조시간(hr)','40대_남성_코로나','전체_맛집','중학교등교여부',
            'season','clust_방문거주2','kclus_카드','dayofweek_num','시_구분','호텔/콘도','30대_코로나',
            '모텔/여관/여인숙','음식','전국_사망자수','전국_격리해제수','kclus_방문유동','kclus_거주연령','전체_날씨',
            '세대당_외식_결제금액','총거주인구','롯데관광주가','화물량','40대_렌트카','40대_버스','학교개수','평균 증기압(hPa)',
            '카페','전체_코로나_lag1','전체_코로나_lag3','전체_코로나_lag5','전체_코로나_lag7','전체_코로나_lag14',
            '전체_숙박_lag1','전체_숙박_lag3','전체_숙박_lag5','전체_숙박_lag7','전체_숙박_lag14','전체_버스_lag1',
            '전체_버스_lag3','전체_버스_lag5','전체_버스_lag7','전체_버스_lag14','전체_맛집_lag1','전체_맛집_lag3',
            '전체_맛집_lag5','전체_맛집_lag7','전체_맛집_lag14','전체_렌트카_lag1','전체_렌트카_lag3','전체_렌트카_lag5',
            '전체_렌트카_lag7','전체_렌트카_lag14','전체_날씨_lag1','전체_날씨_lag3','전체_날씨_lag5','전체_날씨_lag7',
            '전체_날씨_lag14','전체_항공_lag1','전체_항공_lag3','전체_항공_lag5','전체_항공_lag7','전체_항공_lag14',
            '전체_제주코로나_lag1','전체_제주코로나_lag3','전체_제주코로나_lag5','전체_제주코로나_lag7','전체_제주코로나_lag14',
            '평균기온(°C)_lag1','평균기온(°C)_lag3','평균기온(°C)_lag5','평균기온(°C)_lag7','최저기온(°C)_lag1',
            '최저기온(°C)_lag3','최저기온(°C)_lag5','최저기온(°C)_lag7','최고기온(°C)_lag1','최고기온(°C)_lag3',
            '최고기온(°C)_lag5','최고기온(°C)_lag7','강수 계속시간(hr)_lag1','강수 계속시간(hr)_lag3','강수 계속시간(hr)_lag5',
            '강수 계속시간(hr)_lag7','10분 최다 강수량(mm)_lag1','10분 최다 강수량(mm)_lag3','10분 최다 강수량(mm)_lag5',
            '10분 최다 강수량(mm)_lag7','1시간 최다강수량(mm)_lag1','1시간 최다강수량(mm)_lag3','1시간 최다강수량(mm)_lag5',
            '1시간 최다강수량(mm)_lag7','일강수량(mm)_lag1','일강수량(mm)_lag3','일강수량(mm)_lag5','일강수량(mm)_lag7',
            '최대 순간 풍속(m/s)_lag1','최대 순간 풍속(m/s)_lag3','최대 순간 풍속(m/s)_lag5','최대 순간 풍속(m/s)_lag7',
            '최대 풍속(m/s)_lag1','최대 풍속(m/s)_lag3','최대 풍속(m/s)_lag5','최대 풍속(m/s)_lag7','평균 풍속(m/s)_lag1',
            '평균 풍속(m/s)_lag3','평균 풍속(m/s)_lag5','평균 풍속(m/s)_lag7','평균 이슬점온도(°C)_lag1',
            '평균 이슬점온도(°C)_lag3','평균 이슬점온도(°C)_lag5','평균 이슬점온도(°C)_lag7','최소 상대습도(%)_lag1',
            '최소 상대습도(%)_lag3','최소 상대습도(%)_lag5','최소 상대습도(%)_lag7','평균 상대습도(%)_lag1',
            '평균 상대습도(%)_lag3','평균 상대습도(%)_lag5','평균 상대습도(%)_lag7','평균 증기압(hPa)_lag1',
            '평균 증기압(hPa)_lag3','평균 증기압(hPa)_lag5','평균 증기압(hPa)_lag7','평균 현지기압(hPa)_lag1',
            '평균 현지기압(hPa)_lag3','평균 현지기압(hPa)_lag5','평균 현지기압(hPa)_lag7','최고 해면기압(hPa)_lag1',
            '최고 해면기압(hPa)_lag3','최고 해면기압(hPa)_lag5','최고 해면기압(hPa)_lag7','최저 해면기압(hPa)_lag1',
            '최저 해면기압(hPa)_lag3','최저 해면기압(hPa)_lag5','최저 해면기압(hPa)_lag7','평균 해면기압(hPa)_lag1',
            '평균 해면기압(hPa)_lag3','평균 해면기압(hPa)_lag5','평균 해면기압(hPa)_lag7','합계 일조시간(hr)_lag1',
            '합계 일조시간(hr)_lag3','합계 일조시간(hr)_lag5','합계 일조시간(hr)_lag7','1시간 최다일사량(MJ/m2)_lag1',
            '1시간 최다일사량(MJ/m2)_lag3','1시간 최다일사량(MJ/m2)_lag5','1시간 최다일사량(MJ/m2)_lag7',
            '합계 일사량(MJ/m2)_lag1','합계 일사량(MJ/m2)_lag3','합계 일사량(MJ/m2)_lag5','합계 일사량(MJ/m2)_lag7',
            '평균 전운량(1/10)_lag1','평균 전운량(1/10)_lag3','평균 전운량(1/10)_lag5','평균 전운량(1/10)_lag7',
            '평균 중하층운량(1/10)_lag1','평균 중하층운량(1/10)_lag3','평균 중하층운량(1/10)_lag5','평균 중하층운량(1/10)_lag7',
            '합계 대형증발량(mm)_lag1','합계 대형증발량(mm)_lag3','합계 대형증발량(mm)_lag5','합계 대형증발량(mm)_lag7',
            '합계 소형증발량(mm)_lag1','합계 소형증발량(mm)_lag3','합계 소형증발량(mm)_lag5','합계 소형증발량(mm)_lag7',
            '안개 계속시간(hr)_lag1','안개 계속시간(hr)_lag3','안개 계속시간(hr)_lag5','안개 계속시간(hr)_lag7','태풍_lag1',
            '태풍_lag3','태풍_lag5','태풍_lag7','10대_이하_낮_제주_거주인구','20대_낮_제주_거주인구','40_50대_낮_제주_거주인구',
            '60대_이상_낮_제주_거주인구','20대_밤_제주_거주인구','30대_밤_제주_거주인구','40_50대_밤_제주_거주인구',
            '30대_낮_제주_거주인구','세대1','세대2','세대3','세대4이상','1인가구','1인가구비율']


features = list(dict.fromkeys(features))
len(features)

In [ ]:
#cate_fe = {'시_구분','dayofweek_name','season','dateName','period','행정동명','읍면동'}

cate_fe = {'year','dayofweek_name','season','dateName','행정동명','시_구분','period','읍면동','month'}
for col in cate_fe:
    local_map = {}
    for i, loc in enumerate(df[col].unique()):
        local_map[loc] = i
    df[col] = df[col].map(local_map)
    df[col] = df[col].astype(object)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso

x = df[features].fillna(0)

def lasso(df, col):
    
    x = df[features].fillna(0)
    y = df[col]

    X_features = x.values
    X_features_scaled = StandardScaler().fit_transform(X_features)
    print(x.shape)

    lasso_low = Lasso(alpha=0.5, max_iter=1000).fit(X_features_scaled, y)
    print("사용한 인자의 수 : {}".format(np.sum(lasso_low.coef_ != 0)))

    result = pd.DataFrame(lasso_low.coef_, index=features).reset_index()
    result.columns = ['변수', '계수']
    result['계수_절댓값'] = result['계수'].abs()
    result.sort_values(by='계수_절댓값', ascending=False)

    select_cat = result[result['계수']!= 0].변수.unique()
    print(len(select_cat))

    cat_features = ["시_구분", "행정동명",'dayofweek_name','season','dateName','is_weekend','isHoliday',
                'year','month','clust_방문거주', 'clust_방문거주2',
                'kclus_배출', 'kclus_거주연령', 'kclus_시간유동', 'kclus_방문유동', 'kclus_카드소비', 
                'kclus_ratio','kclus_세대수', 'kclus_배출량2','kclus_카드',
                'period','읍면동']
    select_cat = list(set(select_cat).union(set(cat_features)))
    #select_cat = list(set(select_cat).union(set(cat_features)).union(set(cat13)).union(set(cat50)).union(set(cat150)))
    select_cat2 = pd.DataFrame({"변수":select_cat})
    select_cat2.to_csv(f'{col}_selected.csv', index=False)
    print(len(select_cat))
    
    return select_cat

## CatBoost 모델링 (shap values 및 시각화)

In [ ]:
#random seed 지정 
seed = 1997

# Set up folds
K = 10
kf = KFold(n_splits = K, random_state = seed , shuffle = True)
OPTIMIZE_ROUNDS = False 

In [ ]:
cat_features = ["시_구분", "행정동명",'dayofweek_name','season','dateName','is_weekend','isHoliday',
                'year','month','clust_방문거주', 'clust_방문거주2',
                'kclus_배출', 'kclus_거주연령', 'kclus_시간유동', 'kclus_방문유동', 'kclus_카드소비', 
                'kclus_ratio','kclus_세대수', 'kclus_배출량2','kclus_카드',
                'period','읍면동']

# MAE 평가지표 함수 만들기 
def rmse(y, y_pred):
    return sqrt(mean_squared_error(y, y_pred))

def cats(train, test, col, select_cat):
    
    y = train[col]

    ### cat modeling 위한 data지정 
    X_cat = train[select_cat]
    X_test = test[select_cat]

    #cat_features : LabelEncoder
    for data in [X_cat, X_test]:
        data[cat_features] = data[cat_features].astype(str)

    
    #catboost에서는 cat_feature를 파라미터로 지정하는데 이때 범주형 변수가 실수형이라면 돌아가지 않음. 에러 방지를 위해서 문자형으로 바꿔주기 
 

    y_valid_pred = 0*y
    y_test_pred = 0
    X_cat.head()
    
    # Set up 
    cat_model = CatBoostRegressor(
        cat_features=cat_features,
        verbose = False, eval_metric="RMSE"
    )

    # Run CV
    for i, (train_index, test_index) in enumerate(kf.split(X_cat)):

        # Create data for this fold
        y_train, y_valid = y.iloc[train_index], y.iloc[test_index]
        X_train, X_valid = X_cat.iloc[train_index,:], X_cat.iloc[test_index,:]
        print( "\nFold = ", i)

        # Run model for this fold
        if OPTIMIZE_ROUNDS:
            fit_model = cat_model.fit( X_train, y_train, 
                                   eval_set=[(X_valid, y_valid)],
                                   use_best_model=True
                                 )
            print( "  N trees = ", cat_model.tree_count_ )
        else:
            fit_model = cat_model.fit( X_train, y_train)

        # Generate validation predictions for this fold
        pred = fit_model.predict(X_valid)
        print( "  RMSE = ",  sqrt(mean_squared_error(y_valid, pred)))
        
        y_valid_pred.iloc[test_index] = pred

        # Accumulate test set predictions
        y_test_pred += fit_model.predict(X_test)

    y_test_pred /= K  # Average test set predictions

    print( "\nRMSE for full training set:" , sqrt(mean_squared_error(y_valid, pred)))  
    print(f'{col} mean: {df_train[col].mean()}')
    #print("\nRMSE for full test set:" , sqrt(mean_squared_error(test[col],y_test_pred)))
    #print(round(sqrt(mean_squared_error(test[col],y_test_pred)) / df_train[col].mean() * 100, 3), '%')
    
     
    explainer = shap.TreeExplainer(fit_model)
    shap_values = explainer.shap_values(X_cat)
    # visualize the first prediction's explanation
    fig1 = plt.figure(figsize=(30,30))
    shap.summary_plot(shap_values, X_cat, show=False)
    plt.savefig(f'{col}_shap1.png',bbox_inches = 'tight')
    fig2 = plt.figure(figsize=(30,50))
    shap.summary_plot(shap_values, X_cat, plot_type='bar', show=False)
    plt.savefig(f'{col}_shap2.png',bbox_inches = 'tight')
      
    diag = test.copy()
    diag['배출량'] = y_test_pred
    diag.to_csv(f'{col}_submission.csv', index=False)
    joblib.dump(fit_model, f'{col}_submission.pkl')
    
    return diag, fit_model

In [ ]:
def graphs(df_train, df_pred, col, select_cat, fit_model):
    
    #feature importance graph
    cat_imp = pd.DataFrame()
    cat_imp['feature'] = df_pred[select_cat].columns
    cat_imp['imp'] = fit_model.feature_importances_
    cat_imp= cat_imp.sort_values(by="imp",ascending=False)

    fig = plt.figure(figsize=(15,50))
    plt.xticks(rotation = 45 )

    fig.patch.set_facecolor('xkcd:white')
    sns.barplot(x="imp", y="feature", data=cat_imp);
    fig.savefig(f'{col}_imp.png')
    
    
    # 예측값 그래프
    diag = df_train.copy()
    diag["기준일자"] = pd.to_datetime(diag['기준일자'], format='%Y-%m-%d')
    df_pred["기준일자"] = pd.to_datetime(df_pred['기준일자'], format='%Y-%m-%d')
    diag = diag[['기준일자','행정동명', f'{col}']]
    df_pred = df_pred[['기준일자','행정동명', f'{col}']]
    df_new = pd.concat([diag, df_pred], axis=0)
    
    data = df_new.copy()
    f, ax = plt.subplots(nrows=len(data['행정동명'].unique()), ncols=1, figsize=(30, 200))
    
    for i, hjd in enumerate(data['행정동명'].unique()):
        temp1 = data[(data['행정동명']==hjd) & ((data.기준일자>='2018-07-01')&(data.기준일자<'2018-09-01'))]
        temp2 = data[(data['행정동명']==hjd) & ((data.기준일자>='2019-07-01')&(data.기준일자<'2019-09-01'))]
        temp3 = data[(data['행정동명']==hjd) & ((data.기준일자>='2020-07-01')&(data.기준일자<'2020-09-01'))]  
        temp4 = data[(data['행정동명']==hjd) & ((data.기준일자>='2021-07-01')&(data.기준일자<'2021-09-01'))]
        temp5 = data[(data['행정동명']==hjd)]
        temp5 = temp5[(temp5.isin(temp1)==False) & (temp5.isin(temp2)==False) & (temp5.isin(temp3)==False) & (temp5.isin(temp4)==False)] 
        
        sns.lineplot(x=temp5['기준일자'], y=temp5[f'{col}'], ax=ax[i], ci=None, label='True', lw=2, color='#2c7fb8')
        sns.lineplot(x=temp1['기준일자'], y=temp1[f'{col}'], ax=ax[i], ci=None, label='True', lw=2, color='#de2d26')
        sns.lineplot(x=temp2['기준일자'], y=temp2[f'{col}'], ax=ax[i], ci=None, label='True', lw=2, color='#de2d26')
        sns.lineplot(x=temp3['기준일자'], y=temp3[f'{col}'], ax=ax[i], ci=None, label='True', lw=2, color='#de2d26')
        sns.lineplot(x=temp4['기준일자'], y=temp4[f'{col}'], ax=ax[i], ci=None, label='Prediction', lw=2, color='#de2d26')


        f.patch.set_facecolor('xkcd:white')

        ax[i].set_title(f'{hjd}: 2018년 7-8월', fontsize=15)
        ax[i].set_title(f'{hjd}: 2019년 7-8월', fontsize=15)
        ax[i].set_title(f'{hjd}: 2020년 7-8월', fontsize=15)
        ax[i].set_title(f'{hjd}: 2021년 7-8월', fontsize=15)


    plt.tight_layout()
    plt.show()
    f.savefig(f'{col}_pred.png')
    
    ## 행정동별 그래프
    data = df_new.copy()
    fg, ax = plt.subplots(nrows=len(data['행정동명'].unique()), ncols=1, figsize=(10, 200))

    for i, hjd in enumerate(data['행정동명'].unique()):
        temp1 = data[(data['행정동명']==hjd) & (data.기준일자 <'2021-07-01')]
        temp2 = data[(data['행정동명']==hjd) & (data.기준일자>='2021-07-01')] 
        sns.lineplot(x=temp1['기준일자'], y=temp1[f'{col}'], ax=ax[i], ci=None, label='True',color='#2c7fb8')
        sns.lineplot(x=temp2['기준일자'], y=temp2[f'{col}'], ax=ax[i], ci=None, label='Prediction', color='#de2d26')

        ax[i].set_title(f'{hjd}', fontsize=15)

    plt.tight_layout()
    plt.show()
    fg.savefig(f'{col}_hjd.png')
    

In [ ]:
var_set = ['배출량']
for cols in var_set:
    print(cols)
    select_cat = lasso(df,cols)
    df_pred, fit_model = cats(df_train, df_test, cols, select_cat)
    graphs(df_train, df_pred, cols, select_cat, fit_model)

## Model 1(CatBoost) - submission 파일 생성

In [ ]:
#cluster
cluster_cat = ['남원읍', '노형동', '대륜동', '대정읍', '대천동', '봉개동', '송산동', '아라동',
               '안덕면', '애월읍', '이도1동', '이도2동', '효돈동', '구좌읍', '조천읍', '한경면', '한림읍']
df_pred_cat = df_pred[df_pred['행정동명'].isin(cluster_cat)==True]

In [ ]:
sub = df_pred_cat.groupby(['행정동명','month']).sum().reset_index()[['행정동명','month','배출량']]

In [ ]:
sub = sub.pivot(index=['행정동명'], columns='month', values='배출량').reset_index().rename(
    columns = {7 : "7월 배출량(g)", 8 : "8월 배출량(g)"})

In [ ]:
sub.head()

In [ ]:
sub.to_csv('submission_cat.csv', index=False, encoding='cp949')

In [ ]:
submission.isna().sum()

# Model 2. CatBoost + RandomForest

## data 불러오기

In [ ]:
#df_train
df = pd.read_csv('df_ver0914.csv', encoding='cp949')
df["기준일자"] = pd.to_datetime(df['기준일자'], format='%Y-%m-%d')
df['year_month'] = pd.to_datetime(df['year_month'])


#df_test
df_test = pd.read_csv('df_test_pred.csv', encoding='cp949')
df_test["기준일자"] = pd.to_datetime(df_test['기준일자'], format='%Y-%m-%d')

In [ ]:
df = df[df['배출량'].notna()]

In [ ]:
df['dateName'] = df['dateName'].fillna('일반')
df_test['dateName'] = df_test['dateName'].fillna('일반')
df_test['화물량'] = df_test['화물량'].fillna(9104.84)
df = df.fillna(0)
df_test = df_test.fillna(0)

In [ ]:
drop_list = ['행정동_코드', '배출건수', '총_지불금액', '외식_결제건수', '외식_결제금액', 
       '간식_결제건수', '농축수산물_결제건수', '마트/슈퍼마켓_결제건수', '배달_결제건수', '부페_결제건수',
       '식품_결제건수', '아시아음식_결제건수', '양식_결제건수', '주점및주류판매_결제건수', '패스트푸드_결제건수',
       '한식_결제건수', '간식_결제금액', '농축수산물_결제금액', '마트/슈퍼마켓_결제금액', '배달_결제금액',
       '부페_결제금액', '식품_결제금액', '아시아음식_결제금액', '양식_결제금액', '주점및주류판매_결제금액',
       '패스트푸드_결제금액', '한식_결제금액', '외식_결제건수', '외식_결제금액', '1회당_외식_결제금액',
       '1회당_마트/슈퍼마켓_결제금액', '1회당_배달_결제금액', '1회당_농축수산물_결제금액']
df = df.drop(drop_list, axis=1)

In [ ]:
df_train = df.copy()
df1 = df.copy()

## LASSO 변수선택

### 라벨 인코딩

In [ ]:
#cate_fe = {'시_구분','dayofweek_name','season','dateName','period','행정동명','읍면동'}
cate_fe = {'year','dayofweek_name','season','dateName','행정동명','시_구분','period','읍면동','month'}
for col in cate_fe:
    local_map = {}
    for i, loc in enumerate(df1[col].unique()):
        local_map[loc] = i
    df1[col] = df1[col].map(local_map)
    df1[col] = df1[col].astype('category').cat.codes

In [2]:
cat_features = ["시_구분", "행정동명",'dayofweek_name','season','dateName','is_weekend','isHoliday',
                'year','month','clust_방문거주', 'clust_방문거주2',
                'kclus_배출', 'kclus_거주연령', 'kclus_시간유동', 'kclus_방문유동', 'kclus_카드소비', 
                'kclus_ratio','kclus_세대수', 'kclus_배출량2','kclus_카드',
                'period','읍면동']

len(cat_features)

22

## 모델링 시작

In [1]:
features = ['kclus_세대수','kclus_month_코로나','면적','density','읍면동','평균 해면기압(hPa)',
            '30대_항공','30대_맛집','20대_맛집','제주_확진자수','30대_버스','신선식품지수','day','60대_숙박',
            '평균 상대습도(%)','50대_제주코로나','50대_버스','60대_맛집','period','40대_날씨','합계 일사량(MJ/m2)',
            '세대당_농축수산물_결제금액','여성_거주인구','30대_날씨','제주항공주가','전국_격리중_환자수','고등학교등교여부',
            '초등학교등교여부','태풍','제주_사망자수','평균기온(°C)','제주_해외유입수','is_weekend','1시간 최다일사량(MJ/m2)',
            '전체_렌트카','최대 순간 풍속(m/s)','20대_숙박','20대_렌트카','50대_날씨','40대_숙박','세대수','식료품소매',
            'dayofweek_name','dateName','60대_렌트카','20대_코로나','평균 풍속(m/s)','최고 해면기압(hPa)','전국_해외유입수',
            '전체_코로나','60대_제주코로나','중학교개수','40대_맛집','남성_거주인구','isHoliday','행정동명','50대_여성_코로나',
            '30대_렌트카','clust_방문거주','제주_격리중_환자수','평균 전운량(1/10)','40대_여성_코로나','제주_전일대비_증감수',
            '20대_버스','전체_제주코로나','60대_날씨','전체_숙박','kclus_ratio','최저기온(°C)','합계 소형증발량(mm)',
            'kclus_배출','평균 현지기압(hPa)','60대_남성_코로나','50대_맛집','캠프/별장/펜션','세대당_배달_결제건수',
            '합계 대형증발량(mm)','dayofyear','최고기온(°C)','최저 해면기압(hPa)','초등학교개수','전체_항공',
            '평균 중하층운량(1/10)','time_idx','60대_버스','50대_항공','전체_버스','생활물가지수','강수 계속시간(hr)',
            'kclus_결제건수','유스호스텔','대기업본사','최소 상대습도(%)','kclus_카드소비','전국_확진자수','20대_항공',
            'year','20대_제주코로나','60대_여성_코로나','60대_항공','전국_전일대비_증감수','상가','평균 이슬점온도(°C)',
            '학문/교육','month_cos','50대_숙박','숙박','거점지역_개수','민박/하숙','고등학교개수','30대_제주코로나',
            '안개 계속시간(hr)','40대_제주코로나','weeknumber','제주_격리해제수','kclus_배출량2','스타벅스','50대_남성_코로나',
            '40대_항공','30대_숙박','kclus_시간유동','한국인세대수','하나투어주가','일강수량(mm)','캠핑장',
            '세대당_마트/슈퍼마켓_결제금액','50대_렌트카','10분 최다 강수량(mm)','month','1시간 최다강수량(mm)',
            '최대 풍속(m/s)','20대_날씨','합계 일조시간(hr)','40대_남성_코로나','전체_맛집','중학교등교여부',
            'season','clust_방문거주2','kclus_카드','dayofweek_num','시_구분','호텔/콘도','30대_코로나',
            '모텔/여관/여인숙','음식','전국_사망자수','전국_격리해제수','kclus_방문유동','kclus_거주연령','전체_날씨',
            '세대당_외식_결제금액','총거주인구','롯데관광주가','화물량','40대_렌트카','40대_버스','학교개수','평균 증기압(hPa)',
            '카페','전체_코로나_lag1','전체_코로나_lag3','전체_코로나_lag5','전체_코로나_lag7','전체_코로나_lag14',
            '전체_숙박_lag1','전체_숙박_lag3','전체_숙박_lag5','전체_숙박_lag7','전체_숙박_lag14','전체_버스_lag1',
            '전체_버스_lag3','전체_버스_lag5','전체_버스_lag7','전체_버스_lag14','전체_맛집_lag1','전체_맛집_lag3',
            '전체_맛집_lag5','전체_맛집_lag7','전체_맛집_lag14','전체_렌트카_lag1','전체_렌트카_lag3','전체_렌트카_lag5',
            '전체_렌트카_lag7','전체_렌트카_lag14','전체_날씨_lag1','전체_날씨_lag3','전체_날씨_lag5','전체_날씨_lag7',
            '전체_날씨_lag14','전체_항공_lag1','전체_항공_lag3','전체_항공_lag5','전체_항공_lag7','전체_항공_lag14',
            '전체_제주코로나_lag1','전체_제주코로나_lag3','전체_제주코로나_lag5','전체_제주코로나_lag7','전체_제주코로나_lag14',
            '평균기온(°C)_lag1','평균기온(°C)_lag3','평균기온(°C)_lag5','평균기온(°C)_lag7','최저기온(°C)_lag1',
            '최저기온(°C)_lag3','최저기온(°C)_lag5','최저기온(°C)_lag7','최고기온(°C)_lag1','최고기온(°C)_lag3',
            '최고기온(°C)_lag5','최고기온(°C)_lag7','강수 계속시간(hr)_lag1','강수 계속시간(hr)_lag3','강수 계속시간(hr)_lag5',
            '강수 계속시간(hr)_lag7','10분 최다 강수량(mm)_lag1','10분 최다 강수량(mm)_lag3','10분 최다 강수량(mm)_lag5',
            '10분 최다 강수량(mm)_lag7','1시간 최다강수량(mm)_lag1','1시간 최다강수량(mm)_lag3','1시간 최다강수량(mm)_lag5',
            '1시간 최다강수량(mm)_lag7','일강수량(mm)_lag1','일강수량(mm)_lag3','일강수량(mm)_lag5','일강수량(mm)_lag7',
            '최대 순간 풍속(m/s)_lag1','최대 순간 풍속(m/s)_lag3','최대 순간 풍속(m/s)_lag5','최대 순간 풍속(m/s)_lag7',
            '최대 풍속(m/s)_lag1','최대 풍속(m/s)_lag3','최대 풍속(m/s)_lag5','최대 풍속(m/s)_lag7','평균 풍속(m/s)_lag1',
            '평균 풍속(m/s)_lag3','평균 풍속(m/s)_lag5','평균 풍속(m/s)_lag7','평균 이슬점온도(°C)_lag1',
            '평균 이슬점온도(°C)_lag3','평균 이슬점온도(°C)_lag5','평균 이슬점온도(°C)_lag7','최소 상대습도(%)_lag1',
            '최소 상대습도(%)_lag3','최소 상대습도(%)_lag5','최소 상대습도(%)_lag7','평균 상대습도(%)_lag1',
            '평균 상대습도(%)_lag3','평균 상대습도(%)_lag5','평균 상대습도(%)_lag7','평균 증기압(hPa)_lag1',
            '평균 증기압(hPa)_lag3','평균 증기압(hPa)_lag5','평균 증기압(hPa)_lag7','평균 현지기압(hPa)_lag1',
            '평균 현지기압(hPa)_lag3','평균 현지기압(hPa)_lag5','평균 현지기압(hPa)_lag7','최고 해면기압(hPa)_lag1',
            '최고 해면기압(hPa)_lag3','최고 해면기압(hPa)_lag5','최고 해면기압(hPa)_lag7','최저 해면기압(hPa)_lag1',
            '최저 해면기압(hPa)_lag3','최저 해면기압(hPa)_lag5','최저 해면기압(hPa)_lag7','평균 해면기압(hPa)_lag1',
            '평균 해면기압(hPa)_lag3','평균 해면기압(hPa)_lag5','평균 해면기압(hPa)_lag7','합계 일조시간(hr)_lag1',
            '합계 일조시간(hr)_lag3','합계 일조시간(hr)_lag5','합계 일조시간(hr)_lag7','1시간 최다일사량(MJ/m2)_lag1',
            '1시간 최다일사량(MJ/m2)_lag3','1시간 최다일사량(MJ/m2)_lag5','1시간 최다일사량(MJ/m2)_lag7',
            '합계 일사량(MJ/m2)_lag1','합계 일사량(MJ/m2)_lag3','합계 일사량(MJ/m2)_lag5','합계 일사량(MJ/m2)_lag7',
            '평균 전운량(1/10)_lag1','평균 전운량(1/10)_lag3','평균 전운량(1/10)_lag5','평균 전운량(1/10)_lag7',
            '평균 중하층운량(1/10)_lag1','평균 중하층운량(1/10)_lag3','평균 중하층운량(1/10)_lag5','평균 중하층운량(1/10)_lag7',
            '합계 대형증발량(mm)_lag1','합계 대형증발량(mm)_lag3','합계 대형증발량(mm)_lag5','합계 대형증발량(mm)_lag7',
            '합계 소형증발량(mm)_lag1','합계 소형증발량(mm)_lag3','합계 소형증발량(mm)_lag5','합계 소형증발량(mm)_lag7',
            '안개 계속시간(hr)_lag1','안개 계속시간(hr)_lag3','안개 계속시간(hr)_lag5','안개 계속시간(hr)_lag7','태풍_lag1',
            '태풍_lag3','태풍_lag5','태풍_lag7','10대_이하_낮_제주_거주인구','20대_낮_제주_거주인구','40_50대_낮_제주_거주인구',
            '60대_이상_낮_제주_거주인구','20대_밤_제주_거주인구','30대_밤_제주_거주인구','40_50대_밤_제주_거주인구',
            '30대_낮_제주_거주인구','세대1','세대2','세대3','세대4이상','1인가구','1인가구비율']


features = list(dict.fromkeys(features))
len(features)

321

In [ ]:
target = '배출량'
features = [col for col in features if col not in\
            [target,'기준일자','year_month','행정동_코드','배출건수','총_지불금액','1인당_배출량']]

selected_features = lasso(df1,target)

#fit_model, y_test_pred = catboost(train, target, selected_features)

### Correlation 확인

In [ ]:
pd.DataFrame(df1.corr()['배출량'].abs().sort_values(ascending=False))

In [ ]:
corr_df = pd.DataFrame(df1.corr()['배출량'].abs().sort_values(ascending=False))
#corr_df.to_csv('corr.csv', encoding="utf-8-sig")
corr_df.index[1:50]

## 배출량 Modeling

In [ ]:
df_train = df1[selected_features] # 설명변수
df_train['y'] = df1['배출량'] # 타겟변수

df_valid = df_test[selected_features]
## setup의 다양한 옵션 
## https://github.com/pycaret/pycaret/blob/master/pycaret/classification.py

from pycaret.regression import *
setup = setup(df_train, target = 'y',
              categorical_features = ["시_구분", "행정동명",'dayofweek_name','season','dateName','is_weekend',
                                      'isHoliday','year','month','clust_방문거주', 'clust_방문거주2',
                                      'kclus_배출', 'kclus_거주연령', 'kclus_시간유동', 'kclus_방문유동', 'kclus_카드소비',
                                      '태풍','period','읍면동'],
              session_id = 1997
             )

In [ ]:
best = compare_models(sort="RMSE", exclude=['omp','par','xgboost','lasso','lr','lightgbm','gbr','et','huber',
                                            'en','ridge','llar','br','knn','dt','ada'])

In [ ]:
cat= create_model('catboost')

In [ ]:
rf = create_model('rf')

In [ ]:
blend1 = blend_models([cat,rf],optimize='RMSE',verbose=True)

In [ ]:
final_model = finalize_model(blend1)

In [ ]:
y_valid1 = predict_model(final_model, data = df_valid)

In [ ]:
# 예측값 그래프

diag = df.copy()
df_pred = y_valid1.copy()

diag['기준일자'] =diag['time_idx'].apply(lambda x: datetime.fromordinal(x + 736694) )
diag["기준일자"] = pd.to_datetime(diag['기준일자'], format='%Y-%m-%d')
df_pred['기준일자'] =df_pred['time_idx'].apply(lambda x: datetime.fromordinal(x + 736694) )
df_pred["기준일자"] = pd.to_datetime(df_pred['기준일자'], format='%Y-%m-%d')

df_pred['배출량'] = y_valid1['Label']
diag = diag[['기준일자','행정동명', '배출량']]
df_pred = df_pred[['기준일자','행정동명', '배출량']]
df_new = pd.concat([diag, df_pred], axis=0)

data = df_new.copy()
f, ax = plt.subplots(nrows=len(data['행정동명'].unique()), ncols=1, figsize=(30, 100))

for i, hjd in enumerate(data['행정동명'].unique()):
    temp1 = data[(data['행정동명']==hjd) & ((data.기준일자>='2018-07-01')&(data.기준일자<'2018-09-01'))]
    temp2 = data[(data['행정동명']==hjd) & ((data.기준일자>='2019-07-01')&(data.기준일자<'2019-09-01'))]
    temp3 = data[(data['행정동명']==hjd) & ((data.기준일자>='2020-07-01')&(data.기준일자<'2020-09-01'))]  
    temp4 = data[(data['행정동명']==hjd) & ((data.기준일자>='2021-07-01')&(data.기준일자<'2021-09-01'))]
    temp5 = data[(data['행정동명']==hjd)]
    temp5 = temp5[(temp5.isin(temp1)==False) & (temp5.isin(temp2)==False) & (temp5.isin(temp3)==False) & (temp5.isin(temp4)==False)] 

    sns.lineplot(x=temp5['기준일자'], y=temp5['배출량'], ax=ax[i], ci=None, label='True', lw=2, color='#2c7fb8')
    sns.lineplot(x=temp1['기준일자'], y=temp1['배출량'], ax=ax[i], ci=None, label='True', lw=2, color='#de2d26')
    sns.lineplot(x=temp2['기준일자'], y=temp2['배출량'], ax=ax[i], ci=None, label='True', lw=2, color='#de2d26')
    sns.lineplot(x=temp3['기준일자'], y=temp3['배출량'], ax=ax[i], ci=None, label='True', lw=2, color='#de2d26')
    sns.lineplot(x=temp4['기준일자'], y=temp4['배출량'], ax=ax[i], ci=None, label='Prediction', lw=2, color='#de2d26')


    f.patch.set_facecolor('xkcd:white')

    ax[i].set_title(f'{hjd}: 2018년 7-8월', fontsize=15)
    ax[i].set_title(f'{hjd}: 2019년 7-8월', fontsize=15)
    ax[i].set_title(f'{hjd}: 2020년 7-8월', fontsize=15)
    ax[i].set_title(f'{hjd}: 2021년 7-8월', fontsize=15)


plt.tight_layout()
plt.show()
#f.savefig(f'{col}_pred.png')

## 행정동별 그래프
data = df_new.copy()
fg, ax = plt.subplots(nrows=len(data['행정동명'].unique()), ncols=1, figsize=(10, 60))

for i, hjd in enumerate(data['행정동명'].unique()):
    temp1 = data[(data['행정동명']==hjd) & (data.기준일자 <'2021-07-01')]
    temp2 = data[(data['행정동명']==hjd) & (data.기준일자>='2021-07-01')] 
    sns.lineplot(x=temp1['기준일자'], y=temp1['배출량'], ax=ax[i], ci=None, label='True',color='#2c7fb8')
    sns.lineplot(x=temp2['기준일자'], y=temp2['배출량'], ax=ax[i], ci=None, label='Prediction', color='#de2d26')

    ax[i].set_title(f'{hjd}', fontsize=15)

plt.tight_layout()
plt.show()
#fg.savefig(f'{col}_hjd.png')

In [ ]:
## 폰트
from matplotlib import font_manager, rc
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
#interpret_model(catboost)
plot_model(cat, 'feature')

In [ ]:
plot_model(rf, 'feature')

# Model 3. Cluster 모델링

## 클러스터별 모델링

In [ ]:
#df_train
df = pd.read_csv('df_ver0914.csv', encoding='cp949')
df["기준일자"] = pd.to_datetime(df['기준일자'], format='%Y-%m-%d')
df['year_month'] = pd.to_datetime(df['year_month'])


#df_test
df_test = pd.read_csv('df_test_pred.csv', encoding='cp949')
df_test["기준일자"] = pd.to_datetime(df_test['기준일자'], format='%Y-%m-%d')

In [ ]:
df = df[df['배출량'].notna()]

In [ ]:
df['dateName'] = df['dateName'].fillna('일반')
df_test['dateName'] = df_test['dateName'].fillna('일반')
df_test['화물량'] = df_test['화물량'].fillna(9104.84)
df = df.fillna(0)
df_test = df_test.fillna(0)

In [ ]:
drop_list = ['행정동_코드', '배출건수', '총_지불금액', '외식_결제건수', '외식_결제금액', 
       '간식_결제건수', '농축수산물_결제건수', '마트/슈퍼마켓_결제건수', '배달_결제건수', '부페_결제건수',
       '식품_결제건수', '아시아음식_결제건수', '양식_결제건수', '주점및주류판매_결제건수', '패스트푸드_결제건수',
       '한식_결제건수', '간식_결제금액', '농축수산물_결제금액', '마트/슈퍼마켓_결제금액', '배달_결제금액',
       '부페_결제금액', '식품_결제금액', '아시아음식_결제금액', '양식_결제금액', '주점및주류판매_결제금액',
       '패스트푸드_결제금액', '한식_결제금액', '외식_결제건수', '외식_결제금액', '1회당_외식_결제금액',
       '1회당_마트/슈퍼마켓_결제금액', '1회당_배달_결제금액', '1회당_농축수산물_결제금액']
df = df.drop(drop_list, axis=1)

In [ ]:
df1 = df.copy()
cluster_etc = ['성산읍', '표선면', '예래동', '도두동', '중앙동']
df1 = df1[df1['행정동명'].isin(cluster_etc) == True]
df_test1 = df_test[df_test['행정동명'].isin(cluster_etc) == True]
df_train = df1.copy()

## LASSO 변수선택

### 라벨 인코딩

In [ ]:
#cate_fe = {'시_구분','dayofweek_name','season','dateName','period','행정동명','읍면동'}
cate_fe = {'year','dayofweek_name','season','dateName','행정동명','시_구분','period','읍면동','month'}
for col in cate_fe:
    local_map = {}
    for i, loc in enumerate(df1[col].unique()):
        local_map[loc] = i
    df1[col] = df1[col].map(local_map)
    df1[col] = df1[col].astype('category').cat.codes


In [ ]:
cat_features = ["시_구분", "행정동명",'dayofweek_name','season','dateName','is_weekend','isHoliday',
                'year','month','clust_방문거주', 'clust_방문거주2',
                'kclus_배출', 'kclus_거주연령', 'kclus_시간유동', 'kclus_방문유동', 'kclus_카드소비',
                '태풍','period','읍면동']

## 모델링 시작

In [3]:
features = ['kclus_세대수','kclus_month_코로나','면적','density','읍면동','평균 해면기압(hPa)',
            '30대_항공','30대_맛집','20대_맛집','제주_확진자수','30대_버스','신선식품지수','day','60대_숙박',
            '평균 상대습도(%)','50대_제주코로나','50대_버스','60대_맛집','period','40대_날씨','합계 일사량(MJ/m2)',
            '세대당_농축수산물_결제금액','여성_거주인구','30대_날씨','제주항공주가','전국_격리중_환자수','고등학교등교여부',
            '초등학교등교여부','태풍','제주_사망자수','평균기온(°C)','제주_해외유입수','is_weekend','1시간 최다일사량(MJ/m2)',
            '전체_렌트카','최대 순간 풍속(m/s)','20대_숙박','20대_렌트카','50대_날씨','40대_숙박','세대수','식료품소매',
            'dayofweek_name','dateName','60대_렌트카','20대_코로나','평균 풍속(m/s)','최고 해면기압(hPa)','전국_해외유입수',
            '전체_코로나','60대_제주코로나','중학교개수','40대_맛집','남성_거주인구','isHoliday','행정동명','50대_여성_코로나',
            '30대_렌트카','clust_방문거주','제주_격리중_환자수','평균 전운량(1/10)','40대_여성_코로나','제주_전일대비_증감수',
            '20대_버스','전체_제주코로나','60대_날씨','전체_숙박','kclus_ratio','최저기온(°C)','합계 소형증발량(mm)',
            'kclus_배출','평균 현지기압(hPa)','60대_남성_코로나','50대_맛집','캠프/별장/펜션','세대당_배달_결제건수',
            '합계 대형증발량(mm)','dayofyear','최고기온(°C)','최저 해면기압(hPa)','초등학교개수','전체_항공',
            '평균 중하층운량(1/10)','time_idx','60대_버스','50대_항공','전체_버스','생활물가지수','강수 계속시간(hr)',
            'kclus_결제건수','유스호스텔','대기업본사','최소 상대습도(%)','kclus_카드소비','전국_확진자수','20대_항공',
            'year','20대_제주코로나','60대_여성_코로나','60대_항공','전국_전일대비_증감수','상가','평균 이슬점온도(°C)',
            '학문/교육','month_cos','50대_숙박','숙박','거점지역_개수','민박/하숙','고등학교개수','30대_제주코로나',
            '안개 계속시간(hr)','40대_제주코로나','weeknumber','제주_격리해제수','kclus_배출량2','스타벅스','50대_남성_코로나',
            '40대_항공','30대_숙박','kclus_시간유동','한국인세대수','하나투어주가','일강수량(mm)','캠핑장',
            '세대당_마트/슈퍼마켓_결제금액','50대_렌트카','10분 최다 강수량(mm)','month','1시간 최다강수량(mm)',
            '최대 풍속(m/s)','20대_날씨','합계 일조시간(hr)','40대_남성_코로나','전체_맛집','중학교등교여부',
            'season','clust_방문거주2','kclus_카드','dayofweek_num','시_구분','호텔/콘도','30대_코로나',
            '모텔/여관/여인숙','음식','전국_사망자수','전국_격리해제수','kclus_방문유동','kclus_거주연령','전체_날씨',
            '세대당_외식_결제금액','총거주인구','롯데관광주가','화물량','40대_렌트카','40대_버스','학교개수','평균 증기압(hPa)',
            '카페','전체_코로나_lag1','전체_코로나_lag3','전체_코로나_lag5','전체_코로나_lag7','전체_코로나_lag14',
            '전체_숙박_lag1','전체_숙박_lag3','전체_숙박_lag5','전체_숙박_lag7','전체_숙박_lag14','전체_버스_lag1',
            '전체_버스_lag3','전체_버스_lag5','전체_버스_lag7','전체_버스_lag14','전체_맛집_lag1','전체_맛집_lag3',
            '전체_맛집_lag5','전체_맛집_lag7','전체_맛집_lag14','전체_렌트카_lag1','전체_렌트카_lag3','전체_렌트카_lag5',
            '전체_렌트카_lag7','전체_렌트카_lag14','전체_날씨_lag1','전체_날씨_lag3','전체_날씨_lag5','전체_날씨_lag7',
            '전체_날씨_lag14','전체_항공_lag1','전체_항공_lag3','전체_항공_lag5','전체_항공_lag7','전체_항공_lag14',
            '전체_제주코로나_lag1','전체_제주코로나_lag3','전체_제주코로나_lag5','전체_제주코로나_lag7','전체_제주코로나_lag14',
            '평균기온(°C)_lag1','평균기온(°C)_lag3','평균기온(°C)_lag5','평균기온(°C)_lag7','최저기온(°C)_lag1',
            '최저기온(°C)_lag3','최저기온(°C)_lag5','최저기온(°C)_lag7','최고기온(°C)_lag1','최고기온(°C)_lag3',
            '최고기온(°C)_lag5','최고기온(°C)_lag7','강수 계속시간(hr)_lag1','강수 계속시간(hr)_lag3','강수 계속시간(hr)_lag5',
            '강수 계속시간(hr)_lag7','10분 최다 강수량(mm)_lag1','10분 최다 강수량(mm)_lag3','10분 최다 강수량(mm)_lag5',
            '10분 최다 강수량(mm)_lag7','1시간 최다강수량(mm)_lag1','1시간 최다강수량(mm)_lag3','1시간 최다강수량(mm)_lag5',
            '1시간 최다강수량(mm)_lag7','일강수량(mm)_lag1','일강수량(mm)_lag3','일강수량(mm)_lag5','일강수량(mm)_lag7',
            '최대 순간 풍속(m/s)_lag1','최대 순간 풍속(m/s)_lag3','최대 순간 풍속(m/s)_lag5','최대 순간 풍속(m/s)_lag7',
            '최대 풍속(m/s)_lag1','최대 풍속(m/s)_lag3','최대 풍속(m/s)_lag5','최대 풍속(m/s)_lag7','평균 풍속(m/s)_lag1',
            '평균 풍속(m/s)_lag3','평균 풍속(m/s)_lag5','평균 풍속(m/s)_lag7','평균 이슬점온도(°C)_lag1',
            '평균 이슬점온도(°C)_lag3','평균 이슬점온도(°C)_lag5','평균 이슬점온도(°C)_lag7','최소 상대습도(%)_lag1',
            '최소 상대습도(%)_lag3','최소 상대습도(%)_lag5','최소 상대습도(%)_lag7','평균 상대습도(%)_lag1',
            '평균 상대습도(%)_lag3','평균 상대습도(%)_lag5','평균 상대습도(%)_lag7','평균 증기압(hPa)_lag1',
            '평균 증기압(hPa)_lag3','평균 증기압(hPa)_lag5','평균 증기압(hPa)_lag7','평균 현지기압(hPa)_lag1',
            '평균 현지기압(hPa)_lag3','평균 현지기압(hPa)_lag5','평균 현지기압(hPa)_lag7','최고 해면기압(hPa)_lag1',
            '최고 해면기압(hPa)_lag3','최고 해면기압(hPa)_lag5','최고 해면기압(hPa)_lag7','최저 해면기압(hPa)_lag1',
            '최저 해면기압(hPa)_lag3','최저 해면기압(hPa)_lag5','최저 해면기압(hPa)_lag7','평균 해면기압(hPa)_lag1',
            '평균 해면기압(hPa)_lag3','평균 해면기압(hPa)_lag5','평균 해면기압(hPa)_lag7','합계 일조시간(hr)_lag1',
            '합계 일조시간(hr)_lag3','합계 일조시간(hr)_lag5','합계 일조시간(hr)_lag7','1시간 최다일사량(MJ/m2)_lag1',
            '1시간 최다일사량(MJ/m2)_lag3','1시간 최다일사량(MJ/m2)_lag5','1시간 최다일사량(MJ/m2)_lag7',
            '합계 일사량(MJ/m2)_lag1','합계 일사량(MJ/m2)_lag3','합계 일사량(MJ/m2)_lag5','합계 일사량(MJ/m2)_lag7',
            '평균 전운량(1/10)_lag1','평균 전운량(1/10)_lag3','평균 전운량(1/10)_lag5','평균 전운량(1/10)_lag7',
            '평균 중하층운량(1/10)_lag1','평균 중하층운량(1/10)_lag3','평균 중하층운량(1/10)_lag5','평균 중하층운량(1/10)_lag7',
            '합계 대형증발량(mm)_lag1','합계 대형증발량(mm)_lag3','합계 대형증발량(mm)_lag5','합계 대형증발량(mm)_lag7',
            '합계 소형증발량(mm)_lag1','합계 소형증발량(mm)_lag3','합계 소형증발량(mm)_lag5','합계 소형증발량(mm)_lag7',
            '안개 계속시간(hr)_lag1','안개 계속시간(hr)_lag3','안개 계속시간(hr)_lag5','안개 계속시간(hr)_lag7','태풍_lag1',
            '태풍_lag3','태풍_lag5','태풍_lag7','10대_이하_낮_제주_거주인구','20대_낮_제주_거주인구','40_50대_낮_제주_거주인구',
            '60대_이상_낮_제주_거주인구','20대_밤_제주_거주인구','30대_밤_제주_거주인구','40_50대_밤_제주_거주인구',
            '30대_낮_제주_거주인구','세대1','세대2','세대3','세대4이상','1인가구','1인가구비율']


features = list(dict.fromkeys(features))
len(features)

321

In [ ]:

target = '배출량'
features = [col for col in features if col not in\
            [target,'기준일자','year_month','행정동_코드','배출건수','총_지불금액','1인당_배출량']]

selected_features = lasso(df1,target)
#fit_model, y_test_pred = catboost(train, target, selected_features)

### Correlation 확인

In [ ]:
pd.DataFrame(df1.corr()['배출량'].abs().sort_values(ascending=False))

In [ ]:
corr_df = pd.DataFrame(df1.corr()['배출량'].abs().sort_values(ascending=False))
#corr_df.to_csv('corr.csv', encoding="utf-8-sig")
corr_df.index[1:50]

## 배출량 Modeling

In [ ]:
df_train = df1[selected_features] # 설명변수
df_train['y'] = df1['배출량'] # 타겟변수

df_valid = df_test1[selected_features]
## setup의 다양한 옵션 
## https://github.com/pycaret/pycaret/blob/master/pycaret/classification.py

from pycaret.regression import *
setup = setup(df_train, target = 'y',
              categorical_features = ["시_구분", "행정동명",'dayofweek_name','season','dateName','is_weekend',
                                      'isHoliday','year','month','clust_방문거주', 'clust_방문거주2',
                                      'kclus_배출', 'kclus_거주연령', 'kclus_시간유동', 'kclus_방문유동', 'kclus_카드소비',
                                      '태풍','period','읍면동'],
              session_id = 1997
             )

In [ ]:
best = compare_models(sort="RMSE", exclude=['omp','par','xgboost','lasso','lr','lightgbm','gbr','et','huber',
                                            'en','ridge','llar','br','knn','dt','ada'])

In [ ]:
cat= create_model('catboost')

In [ ]:
rf = create_model('rf')

In [ ]:
blend1 = blend_models([cat,rf],optimize='RMSE',verbose=True)

In [ ]:
final_model = finalize_model(blend1)

In [ ]:
y_valid = predict_model(final_model, data = df_valid)

In [ ]:
subm = y_valid.groupby(['행정동명','year','month']).sum().reset_index()[['행정동명','month','Label']]

In [ ]:
subm = subm.pivot(index=['행정동명'], columns='month', values='Label').reset_index().rename(columns = {'Label':'배출량', 
                                                                                                   7 : "7월 배출량(g)",
                                                                                                   8 :"8월 배출량(g)"})
                                                                                                   


In [ ]:
subm.to_csv('submission_etc.csv',index=False, encoding='cp949')

In [ ]:
y_valid['기준일자'] =y_valid['time_idx'].apply(lambda x: datetime.fromordinal(x + 736694) )
y_valid["기준일자"] = pd.to_datetime(diag['기준일자'], format='%Y-%m-%d')


In [ ]:
# 예측값 그래프
df2= df.copy()
cluster_etc = ['성산읍', '표선면', '예래동', '도두동', '중앙동']
df2 = df2[df2['행정동명'].isin(cluster_etc) == True]
diag = df2.copy()
df_pred = y_valid.copy()

diag['기준일자'] =diag['time_idx'].apply(lambda x: datetime.fromordinal(x + 736694) )
diag["기준일자"] = pd.to_datetime(diag['기준일자'], format='%Y-%m-%d')
df_pred['기준일자'] =df_pred['time_idx'].apply(lambda x: datetime.fromordinal(x + 736694) )
df_pred["기준일자"] = pd.to_datetime(df_pred['기준일자'], format='%Y-%m-%d')

df_pred['배출량'] = y_valid['Label']
diag = diag[['기준일자','행정동명', '배출량']]
df_pred = df_pred[['기준일자','행정동명', '배출량']]
df_new = pd.concat([diag, df_pred], axis=0)


In [ ]:
df_new.shape

In [ ]:
# 예측값 그래프
df2= df.copy()
cluster_etc = ['성산읍', '표선면', '예래동', '도두동', '중앙동']
df2 = df2[df2['행정동명'].isin(cluster_etc) == True]
diag = df2.copy()
df_pred = y_valid.copy()

diag['기준일자'] =diag['time_idx'].apply(lambda x: datetime.fromordinal(x + 736694) )
diag["기준일자"] = pd.to_datetime(diag['기준일자'], format='%Y-%m-%d')
df_pred['기준일자'] =df_pred['time_idx'].apply(lambda x: datetime.fromordinal(x + 736694) )
df_pred["기준일자"] = pd.to_datetime(df_pred['기준일자'], format='%Y-%m-%d')

df_pred['배출량'] = y_valid['Label']
diag = diag[['기준일자','행정동명', '배출량']]
df_pred = df_pred[['기준일자','행정동명', '배출량']]
df_new = pd.concat([diag, df_pred], axis=0)

data = df_new.copy()
f, ax = plt.subplots(nrows=len(data['행정동명'].unique()), ncols=1, figsize=(30, 50))

for i, hjd in enumerate(data['행정동명'].unique()):
    temp1 = data[(data['행정동명']==hjd) & ((data.기준일자>='2018-07-01')&(data.기준일자<'2018-09-01'))]
    temp2 = data[(data['행정동명']==hjd) & ((data.기준일자>='2019-07-01')&(data.기준일자<'2019-09-01'))]
    temp3 = data[(data['행정동명']==hjd) & ((data.기준일자>='2020-07-01')&(data.기준일자<'2020-09-01'))]  
    temp4 = data[(data['행정동명']==hjd) & ((data.기준일자>='2021-07-01')&(data.기준일자<'2021-09-01'))]
    temp5 = data[(data['행정동명']==hjd)]
    temp5 = temp5[(temp5.isin(temp1)==False) & (temp5.isin(temp2)==False) & (temp5.isin(temp3)==False) & (temp5.isin(temp4)==False)] 

    sns.lineplot(x=temp5['기준일자'], y=temp5['배출량'], ax=ax[i], ci=None, label='True', lw=2, color='#2c7fb8')
    sns.lineplot(x=temp1['기준일자'], y=temp1['배출량'], ax=ax[i], ci=None, label='True', lw=2, color='#de2d26')
    sns.lineplot(x=temp2['기준일자'], y=temp2['배출량'], ax=ax[i], ci=None, label='True', lw=2, color='#de2d26')
    sns.lineplot(x=temp3['기준일자'], y=temp3['배출량'], ax=ax[i], ci=None, label='True', lw=2, color='#de2d26')
    sns.lineplot(x=temp4['기준일자'], y=temp4['배출량'], ax=ax[i], ci=None, label='Prediction', lw=2, color='#de2d26')


    f.patch.set_facecolor('xkcd:white')

    ax[i].set_title(f'{hjd}: 2018년 7-8월', fontsize=15)
    ax[i].set_title(f'{hjd}: 2019년 7-8월', fontsize=15)
    ax[i].set_title(f'{hjd}: 2020년 7-8월', fontsize=15)
    ax[i].set_title(f'{hjd}: 2021년 7-8월', fontsize=15)


plt.tight_layout()
plt.show()
#f.savefig(f'{col}_pred.png')

## 행정동별 그래프
data = df_new.copy()
fg, ax = plt.subplots(nrows=len(data['행정동명'].unique()), ncols=1, figsize=(10, 60))

for i, hjd in enumerate(data['행정동명'].unique()):
    temp1 = data[(data['행정동명']==hjd) & (data.기준일자 <'2021-07-01')]
    temp2 = data[(data['행정동명']==hjd) & (data.기준일자>='2021-07-01')] 
    sns.lineplot(x=temp1['기준일자'], y=temp1['배출량'], ax=ax[i], ci=None, label='True',color='#2c7fb8')
    sns.lineplot(x=temp2['기준일자'], y=temp2['배출량'], ax=ax[i], ci=None, label='Prediction', color='#de2d26')

    ax[i].set_title(f'{hjd}', fontsize=15)

plt.tight_layout()
plt.show()
#fg.savefig(f'{col}_hjd.png')


In [ ]:
## 폰트
from matplotlib import font_manager, rc
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
#interpret_model(catboost)
plot_model(cat, 'feature')

In [ ]:
plot_model(rf, 'feature')